# **Descripción general del proyecto**

Este proyecto tiene como objetivo desarrollar un **pipeline ETL completo** para el análisis del consumo de energía eléctrica en una cadena de tiendas físicas en distintos países de Latinoamérica, utilizando datos operativos reales simulados a nivel diario.

A lo largo del proyecto se trabaja con un dataset de alta granularidad que contiene información sobre consumo energético, demanda eléctrica, características físicas de las tiendas y su estado operativo. El enfoque principal es **limpiar, transformar y enriquecer los datos**, garantizando su calidad antes de realizar cualquier análisis agregado.

El proyecto se implementa utilizando Pandas, PySpark y SQL, permitiendo comparar enfoques y reforzar el entendimiento de los principios fundamentales de ETL, tales como:

* orden correcto del flujo de transformación,
* manejo de valores nulos con criterio de negocio,
* creación de métricas derivadas relevantes,
* uso adecuado de filtros antes y después de agregaciones.

Este trabajo está diseñado tanto como ejercicio de aprendizaje profundo como **proyecto demostrable de portafolio** orientado a análisis de datos y data engineering.

## Contexto del proyecto

En este notebook se desarrolla un proceso de **ETL y transformación de datos utilizando Pandas**, aplicado a un conjunto de datos de consumo energético diario de tiendas físicas.

El objetivo es preparar un dataset limpio y coherente a partir de datos crudos, abordando problemas comunes en entornos reales como:

* registros de tiendas cerradas,
* valores nulos en variables operativas críticas,
* necesidad de imputación basada en características del negocio.

Durante el proceso se crean columnas derivadas que permiten evaluar indicadores clave de eficiencia energética, como el consumo y la demanda eléctrica normalizados por área de la tienda.
El enfoque prioriza **claridad, orden lógico y reproducibilidad**, siguiendo buenas prácticas de análisis de datos en Python.

In [0]:
import pandas as pd


# CARGA DEL DATASET
df = pd.read_csv(
    '/Volumes/workspace/default/raw_data/consumo_energia_tiendas_diario_2023_2024.csv'
)


# ESQUEMA DEL SET DE DATOS
print("Esquema:")
print(df.head(10))


# INFORMACIÓN DEL DATASET
print(df.info())

print("\nColumnas del set de datos:")
print(list(df.columns))

print("\nNulos en columnas")
print(df.isna().sum())


'''
Aunque las tiendas presentan consumo eléctrico en días cerrados debido a cargas base y sistemas auxiliares, estos registros no representan operación comercial normal. Para evitar distorsionar métricas de eficiencia energética, se excluyen de los análisis operativos.

Existen 5482 registros nulos en las columnas de area_m2, consumo_kwh y demanda_kw. Ya que es muy importante conservar la información completa 
de estas columnas, imputaremos los valores utilizando el promedio de los datos, considerando el tipo de tienda y el país donde se encuentran 
localizadas. 
'''


# FILTRADO DE TIENDAS ACTIVAS
df_active = df[df['estado_tienda'] == 'Activa']


# DETERMINAMOS PROMEDIO DE DATOS
# Área
promedio = (
    df_active
        .groupby(['pais', 'tipo_tienda'])
        .agg(
            promedio_area_m2=('area_m2', 'mean'),
            promedio_consumo_kwh=('consumo_kwh', 'mean'),
            promedio_demanda_kw=('demanda_kw', 'mean')
        )
        .reset_index()
)

print(promedio)
print("\n")


# UNIMOS PROMEDIOS AL DATASET ACTIVO
df_active = df_active.merge(
    promedio,
    on=['pais', 'tipo_tienda'],
    how='left'
)


# IMPUTACIÓN DE NULOS
df_active['area_m2'] = df_active['area_m2'].fillna(df_active['promedio_area_m2'])
df_active['consumo_kwh'] = df_active['consumo_kwh'].fillna(df_active['promedio_consumo_kwh'])
df_active['demanda_kw'] = df_active['demanda_kw'].fillna(df_active['promedio_demanda_kw'])


# ELIMINAMOS COLUMNAS AUXILIARES
df_active = df_active.drop(
    ['promedio_area_m2', 'promedio_consumo_kwh', 'promedio_demanda_kw'],
    axis=1
)

print(df_active.isna().sum())


# CREACIÓN DE KPIs
df_active['kpi_kwh/m2'] = df_active['consumo_kwh'] / df_active['area_m2']
df_active['kpi_kw/m2'] = df_active['demanda_kw'] / df_active['area_m2']


In [0]:
df.head(10)

In [0]:
df_active.head(10)